In [27]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir /kaggle
! cp kaggle.json /kaggle
! chmod 600 /kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d johnsmith88/heart-disease-dataset
! unzip heart-disease-dataset.zip

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/kaggle’: File exists
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Dataset URL: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset
License(s): unknown
heart-disease-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  heart-disease-dataset.zip
replace heart.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: heart

In [30]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def load_data():
    # Load dataset (replace with actual dataset path)
    df = pd.read_csv("heart.csv")

    # Selecting 9 features for better predictions
    X = df[['age', 'sex', 'trestbps', 'chol', 'cp', 'thalach', 'fbs', 'restecg', 'exang']]
    y = df['target']  # Target variable (1: Disease, 0: No Disease)

    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_model(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Save model
    joblib.dump(model, "heart_disease_model.pkl")
    return model

def load_model():
    return joblib.load("heart_disease_model.pkl")

def categorize_blood_pressure(bp):
    if bp < 90:
        return "Low"
    elif 90 <= bp <= 120:
        return "Normal"
    else:
        return "High"

def categorize_cholesterol(chol):
    if chol < 200:
        return "Normal"
    elif 200 <= chol <= 240:
        return "Borderline High"
    else:
        return "High"

def get_user_input():
    print("\n--- Enter Your Health Details ---")
    age = int(input("Age: "))
    sex = int(input("Sex (1=Male, 0=Female): "))

    # Display proper limits near inputs
    trestbps = int(input("Resting Blood Pressure (Low <90, Normal 90-120, High >120): "))
    chol = int(input("Cholesterol Level (Normal <200, Borderline High 200-240, High >240): "))

    cp = int(input("Chest Pain Type (0=None, 1=Mild, 2=Moderate, 3=Severe): "))
    thalach = int(input("Max Heart Rate Achieved (Lower is riskier): "))

    fbs = int(input("Fasting Blood Sugar (>120 mg/dl is risky) (1=Yes, 0=No): "))
    restecg = int(input("Resting ECG Result (0=Normal, 1=ST-T wave abnormality, 2=Possible LVH): "))
    exang = int(input("Exercise-Induced Angina (1=Yes, 0=No): "))

    bp_category = categorize_blood_pressure(trestbps)
    chol_category = categorize_cholesterol(chol)

    return np.array([[age, sex, trestbps, chol, cp, thalach, fbs, restecg, exang]]), bp_category, chol_category, trestbps, chol, cp, exang

def provide_recommendations(risk_level, bp_category, chol_category):
    recommendations = {
        "High": [
            "⚠️ You are at **high risk**! Consult a doctor immediately.",
            "🥗 Eat a diet low in saturated fats and high in fiber.",
            "🏃‍♂️ Exercise at least **30 minutes daily**.",
            "🧘 Reduce stress through meditation or yoga.",
            "🚭 Quit smoking and limit alcohol intake.",
            "💉 Regular checkups for blood pressure, cholesterol, and diabetes."
        ],
        "No Risk, Healthy": [
            "✅ You are **healthy**! Keep up the good lifestyle.",
            "💪 Exercise regularly and maintain a **balanced diet**.",
            "🩺 Occasionally monitor blood pressure and cholesterol.",
            "🚶 Stay active to **prevent future heart risks**."
        ]
    }

    print("\n--- Health Analysis ---")
    print(f"🩸 Blood Pressure Level: {bp_category}")
    print(f"🧪 Cholesterol Level: {chol_category}")

    print("\n--- Risk Prediction ---")
    print(f"💡 Risk Level: {risk_level}")

    print("\n--- Recommended Health Tips ---")
    for tip in recommendations[risk_level]:
        print(f"- {tip}")

def chatbot():
    print("Welcome to the AI-Driven Heart Disease Chatbot!")
    model = load_model()

    user_input, bp_category, chol_category, trestbps, chol, cp, exang = get_user_input()

    # Manual override for high-risk conditions
    if trestbps > 120 or chol > 240 or cp >= 2 or exang == 1:
        risk_level = "High"
    else:
        prediction = model.predict(user_input)[0]
        risk_level = "High" if prediction == 1 else "No Risk, Healthy"

    provide_recommendations(risk_level, bp_category, chol_category)

if __name__ == "__main__":
    X_train, X_test, y_train, y_test = load_data()
    train_model(X_train, X_test, y_train, y_test)
    chatbot()


Welcome to the AI-Driven Heart Disease Chatbot!

--- Enter Your Health Details ---
Age: 21
Sex (1=Male, 0=Female): 1
Resting Blood Pressure (Low <90, Normal 90-120, High >120): 95
Cholesterol Level (Normal <200, Borderline High 200-240, High >240): 150
Chest Pain Type (0=None, 1=Mild, 2=Moderate, 3=Severe): 0
Max Heart Rate Achieved (Lower is riskier): 120
Fasting Blood Sugar (>120 mg/dl is risky) (1=Yes, 0=No): 0
Resting ECG Result (0=Normal, 1=ST-T wave abnormality, 2=Possible LVH): 0
Exercise-Induced Angina (1=Yes, 0=No): 0

--- Health Analysis ---
🩸 Blood Pressure Level: Normal
🧪 Cholesterol Level: Normal

--- Risk Prediction ---
💡 Risk Level: No Risk, Healthy

--- Recommended Health Tips ---
- ✅ You are **healthy**! Keep up the good lifestyle.
- 💪 Exercise regularly and maintain a **balanced diet**.
- 🩺 Occasionally monitor blood pressure and cholesterol.
- 🚶 Stay active to **prevent future heart risks**.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [31]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def load_data():
    # Load dataset (replace with actual dataset path)
    df = pd.read_csv("heart.csv")

    # Selecting 9 features for better predictions
    X = df[['age', 'sex', 'trestbps', 'chol', 'cp', 'thalach', 'fbs', 'restecg', 'exang']]
    y = df['target']  # Target variable (1: Disease, 0: No Disease)

    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_model(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Save model
    joblib.dump(model, "heart_disease_model.pkl")
    return model

def load_model():
    return joblib.load("heart_disease_model.pkl")

def categorize_blood_pressure(bp):
    if bp < 90:
        return "Low"
    elif 90 <= bp <= 120:
        return "Normal"
    else:
        return "High"

def categorize_cholesterol(chol):
    if chol < 200:
        return "Normal"
    elif 200 <= chol <= 240:
        return "Borderline High"
    else:
        return "High"

def get_user_input():
    print("\n--- Enter Your Health Details ---")
    age = int(input("Age: "))
    sex = int(input("Sex (1=Male, 0=Female): "))

    # Display proper limits near inputs
    trestbps = int(input("Resting Blood Pressure (Low <90, Normal 90-120, High >120): "))
    chol = int(input("Cholesterol Level (Normal <200, Borderline High 200-240, High >240): "))

    cp = int(input("Chest Pain Type (0=None, 1=Mild, 2=Moderate, 3=Severe): "))
    thalach = int(input("Max Heart Rate Achieved (Lower is riskier): "))

    fbs = int(input("Fasting Blood Sugar (>120 mg/dl is risky) (1=Yes, 0=No): "))
    restecg = int(input("Resting ECG Result (0=Normal, 1=ST-T wave abnormality, 2=Possible LVH): "))
    exang = int(input("Exercise-Induced Angina (1=Yes, 0=No): "))

    bp_category = categorize_blood_pressure(trestbps)
    chol_category = categorize_cholesterol(chol)

    return np.array([[age, sex, trestbps, chol, cp, thalach, fbs, restecg, exang]]), bp_category, chol_category, trestbps, chol, cp, exang

def provide_recommendations(risk_level, bp_category, chol_category):
    recommendations = {
        "High": [
            "⚠️ You are at **high risk**! Consult a doctor immediately.",
            "🥗 Eat a diet low in saturated fats and high in fiber.",
            "🏃‍♂️ Exercise at least **30 minutes daily**.",
            "🧘 Reduce stress through meditation or yoga.",
            "🚭 Quit smoking and limit alcohol intake.",
            "💉 Regular checkups for blood pressure, cholesterol, and diabetes."
        ],
        "No Risk, Healthy": [
            "✅ You are **healthy**! Keep up the good lifestyle.",
            "💪 Exercise regularly and maintain a **balanced diet**.",
            "🩺 Occasionally monitor blood pressure and cholesterol.",
            "🚶 Stay active to **prevent future heart risks**."
        ]
    }

    print("\n--- Health Analysis ---")
    print(f"🩸 Blood Pressure Level: {bp_category}")
    print(f"🧪 Cholesterol Level: {chol_category}")

    print("\n--- Risk Prediction ---")
    print(f"💡 Risk Level: {risk_level}")

    print("\n--- Recommended Health Tips ---")
    for tip in recommendations[risk_level]:
        print(f"- {tip}")

def chatbot():
    print("Welcome to the AI-Driven Heart Disease Chatbot!")
    model = load_model()

    user_input, bp_category, chol_category, trestbps, chol, cp, exang = get_user_input()

    # Manual override for high-risk conditions
    if trestbps > 120 or chol > 240 or cp >= 2 or exang == 1:
        risk_level = "High"
    else:
        prediction = model.predict(user_input)[0]
        risk_level = "High" if prediction == 1 else "No Risk, Healthy"

    provide_recommendations(risk_level, bp_category, chol_category)

if __name__ == "__main__":
    X_train, X_test, y_train, y_test = load_data()
    train_model(X_train, X_test, y_train, y_test)
    chatbot()


Welcome to the AI-Driven Heart Disease Chatbot!

--- Enter Your Health Details ---
Age: 50
Sex (1=Male, 0=Female): 1
Resting Blood Pressure (Low <90, Normal 90-120, High >120): 110
Cholesterol Level (Normal <200, Borderline High 200-240, High >240): 200
Chest Pain Type (0=None, 1=Mild, 2=Moderate, 3=Severe): 1
Max Heart Rate Achieved (Lower is riskier): 100
Fasting Blood Sugar (>120 mg/dl is risky) (1=Yes, 0=No): 0
Resting ECG Result (0=Normal, 1=ST-T wave abnormality, 2=Possible LVH): 0
Exercise-Induced Angina (1=Yes, 0=No): 0

--- Health Analysis ---
🩸 Blood Pressure Level: Normal
🧪 Cholesterol Level: Borderline High

--- Risk Prediction ---
💡 Risk Level: High

--- Recommended Health Tips ---
- ⚠️ You are at **high risk**! Consult a doctor immediately.
- 🥗 Eat a diet low in saturated fats and high in fiber.
- 🏃‍♂️ Exercise at least **30 minutes daily**.
- 🧘 Reduce stress through meditation or yoga.
- 🚭 Quit smoking and limit alcohol intake.
- 💉 Regular checkups for blood pressure, ch

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
